In [1]:
##############################################################################
#                                                                            #
#  Code for the USENIX Security '22 paper:                                   #
#  How Machine Learning Is Solving the Binary Function Similarity Problem.   #
#                                                                            #
#  MIT License                                                               #
#                                                                            #
#  Copyright (c) 2019-2022 Cisco Talos                                       #
#                                                                            #
#  Permission is hereby granted, free of charge, to any person obtaining     #
#  a copy of this software and associated documentation files (the           #
#  "Software"), to deal in the Software without restriction, including       #
#  without limitation the rights to use, copy, modify, merge, publish,       #
#  distribute, sublicense, and/or sell copies of the Software, and to        #
#  permit persons to whom the Software is furnished to do so, subject to     #
#  the following conditions:                                                 #
#                                                                            #
#  The above copyright notice and this permission notice shall be            #
#  included in all copies or substantial portions of the Software.           #
#                                                                            #
#  THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,           #
#  EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF        #
#  MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND                     #
#  NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE    #
#  LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION    #
#  OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION     #
#  WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.           #
#                                                                            #
#  Dataset-Vulnerability creation                                            #
#                                                                            #
##############################################################################

In [2]:
## Requirements
# tqdm==4.64.0
# pandas==1.4.2

In [3]:
import json
import pandas as pd
import itertools

from tqdm import tqdm

**Read the flowchart CSV**

In [4]:
flowchart = pd.read_csv("../../DBs/Dataset-Muaz/features/flowchart_Dataset-Muaz.csv")
print(flowchart.shape)

(11734, 8)


In [5]:
flowchart.head()

,idb_path,fva,func_name,start_ea,end_ea,bb_num,bb_list,hashopcodes
0,IDBs/Dataset-Muaz/coreutils-5.93-O3-mkdir.i64,0x4012d0,main,0x4012d0,0x40165d,43,0x4012d0;0x401340;0x401360;0x401369;0x40136b;0...,70671fb76e510df13e84b4d93e2d0294146a12f5334b65...
1,IDBs/Dataset-Muaz/coreutils-5.93-O3-mkdir.i64,0x401730,frame_dummy,0x401730,0x401756,5,0x401730;0x40173b;0x401740;0x40174a;0x4016d0,6ca4769ee079b25ccb0a75176d5669906b61204700c2d9...
2,IDBs/Dataset-Muaz/coreutils-5.93-O3-mkdir.i64,0x4018b0,close_stdout,0x4018b0,0x40197b,12,0x4018b0;0x4018d8;0x4018dc;0x4018e8;0x4018ec;0...,3504c8d5de68a34913d7ca85ca65d51ba6a65b529f2e69...
3,IDBs/Dataset-Muaz/coreutils-5.93-O3-mkdir.i64,0x401980,make_dir_parents,0x401980,0x4021d0,89,0x401980;0x4019f4;0x401a0a;0x401a17;0x401a1f;0...,40c714ed9dd5b8be5d2213336b0f14ccbfb956f45632ff...
4,IDBs/Dataset-Muaz/coreutils-5.93-O3-mkdir.i64,0x4021d0,mode_compile,0x4021d0,0x4024af,50,0x4021d0;0x4021e7;0x4021f3;0x4021f8;0x40221c;0...,b6113539d8db45cd6677751997d513eb4cab279844e85d...


In [6]:
flowchart.groupby(['func_name']).count()

,idb_path,fva,start_ea,end_ea,bb_num,bb_list,hashopcodes
func_name,,,,,,,
.__isoc99_scanf,6,6,6,6,6,6,6
.__stack_chk_fail,21,21,21,21,21,21,21
.init_proc,21,21,21,21,21,21,21
.mmap,9,9,9,9,9,9,9
.printf,21,21,21,21,21,21,21
...,...,...,...,...,...,...,...
year,6,6,6,6,6,6,6
yesno,20,20,20,20,20,20,20
yylex,2,2,2,2,2,2,2


In [10]:
flowchart.loc[flowchart['func_name'] == 'getMatrixElements']

,idb_path,fva,func_name,start_ea,end_ea,bb_num,bb_list,hashopcodes
11254,IDBs/Dataset-Muaz/matmul_sum.i64,0x11e9,getMatrixElements,0x11e9,0x12a3,7,0x11e9;0x1217;0x1220;0x1287;0x128f;0x1293;0x129f,edd5c940305dba4e826f2d12169466101c31fbce60d8f9...
11272,IDBs/Dataset-Muaz/matmul_sum.i64,0x11e9,getMatrixElements,0x11e9,0x12a3,7,0x11e9;0x1217;0x1220;0x1287;0x128f;0x1293;0x129f,edd5c940305dba4e826f2d12169466101c31fbce60d8f9...
11356,IDBs/Dataset-Muaz/matmul_sum.i64,0x11e9,getMatrixElements,0x11e9,0x12a3,7,0x11e9;0x1217;0x1220;0x1287;0x128f;0x1293;0x129f,edd5c940305dba4e826f2d12169466101c31fbce60d8f9...
11481,IDBs/Dataset-Muaz/matmul_sum.i64,0x11e9,getMatrixElements,0x11e9,0x12a3,7,0x11e9;0x1217;0x1220;0x1287;0x128f;0x1293;0x129f,edd5c940305dba4e826f2d12169466101c31fbce60d8f9...
11611,IDBs/Dataset-Muaz/matmul_og.i64,0x11c9,getMatrixElements,0x11c9,0x11f2,1,0x11c9,52c8c34fb519a9a979d273152cc372d401e809097e15c4...
11644,IDBs/Dataset-Muaz/matmul_sum.i64,0x11e9,getMatrixElements,0x11e9,0x12a3,7,0x11e9;0x1217;0x1220;0x1287;0x128f;0x1293;0x129f,edd5c940305dba4e826f2d12169466101c31fbce60d8f9...


**Functions of interest**

In [ ]:
fun_of_interest = list(flowchart['func_name'])
fun_of_interest = [
    'slist_wc_append',
    'ssl_srp_verify_param_cb',
    'notef',
    'app_create_libctx',
    'tool_set_stderr'
]
selected_columns = ['idb_path', 'fva', 'func_name', 'hashopcodes']

df0 = flowchart[selected_columns]
df = df0.loc[df0['func_name'].isin(fun_of_interest)]


# Store the new function pairs
df.reset_index(inplace=True)
pairs = list(itertools.combinations(df.index,2))
df = df.drop('index', axis=1)

**Create all pairs of all functions of interest**

In [ ]:
comparison_list = list()
print("Nb of fun: {}".format(len(df)))

# Iterate over each unique pair of function in the list

for f1,f2 in tqdm(set(pairs)):
    comparison_list.append(list(df.iloc[f1]) + list(df.iloc[f2]))

In [ ]:
comparison_list[0]

In [ ]:
len(comparison_list)

In [ ]:
# Create a new DataFrame
columns = [x + "_1" for x in selected_columns ] + [x + "_2" for x in selected_columns ]
testing = pd.DataFrame(comparison_list, columns=columns)

# Add the db_type column 
testing['db_type'] = ['XM'] * testing.shape[0]
print(testing.shape)

In [ ]:
# Sort the rows
testing.sort_values(by=['idb_path_1', 'fva_1', 'idb_path_2', 'fva_2'], inplace=True)
testing.reset_index(inplace=True, drop=True)
print(testing.shape)

In [ ]:
# Check that the hashopcodes of the functions to compare are different
for i, row in testing.iterrows():
    if row['hashopcodes_1'] == row['hashopcodes_2']:
        print("MATCH!")
        print(row)

In [ ]:
# Paranoid check
testing.drop_duplicates(inplace=True)
testing.reset_index(inplace=True, drop=True)
print(testing.shape)

In [ ]:
# Remove hashopcodes columns
del testing['hashopcodes_1']
del testing['hashopcodes_2']

In [ ]:
testing.head()

In [ ]:
testing.tail()

In [ ]:
testing.groupby(['idb_path_1', 'func_name_1']).count()

In [ ]:
testing.groupby(['idb_path_1']).count()

In [ ]:
testing.groupby(['idb_path_2']).count()

In [ ]:
# Save the DataFrame to file
testing.to_csv("../../DBs/Dataset-Muaz/pairs/pairs_testing_Dataset-Muaz.csv")

In [ ]:
# Save the "selected functions" to a JSON.
# This is useful to limit the IDA analysis to some functions only.

testing_functions = set([tuple(x) for x in testing[['idb_path_1', 'fva_1']].values])
testing_functions |= set([tuple(x) for x in testing[['idb_path_2', 'fva_2']].values])
print("Found {} unique functions".format(len(testing_functions)))

from collections import defaultdict
selected_functions = defaultdict(list)
for t in testing_functions:
    selected_functions[t[0]].append(int(t[1], 16))

# Test
assert(sum([len(v) for v in selected_functions.values()]) == len(testing_functions))

# Save to file
with open("../../DBs/Dataset-Muaz/features/selected_Dataset-Muaz.json", "w") as f_out:
    json.dump(selected_functions, f_out)

In [ ]:
# Save the "selected functions" to a CSV.
# This will be useful to post-process the results.

# Remove from flowchart the functions that are not used for the testing
dataset = flowchart.copy()
del dataset['bb_list']
del_list = list()
for i, row in dataset.iterrows():
    if not tuple([row['idb_path'], row['fva']]) in testing_functions:
        del_list.append(i)
dataset.drop(del_list, inplace=True)
dataset.reset_index(inplace=True, drop=True)
print(dataset.shape)

# Save to file
dataset.to_csv("../../DBs/Dataset-Muaz/testing_Dataset-Muaz.csv")